In [1]:
#Import Packages

from arango import ArangoClient
from IPython.display import clear_output
import getpass
import pandas as pd

#Web Scraping Packages
import requests
from requests.utils import requote_uri
from fake_useragent import UserAgent
from lxml import html
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

In [3]:
# Database Connection

username = input('Please enter your username to connect DB: ')
password = getpass.getpass('Please enter your password to connect DB: ')
try:
    client = ArangoClient(hosts='http://localhost:8529')
    FBW = client.db('Flemish_Business_Websites', username=username, password=password)
    clear_output(wait=True)
    print('Successfully connected to the ArangoDB')
    del password
except ConnectionError:
    clear_output(wait=True)
    print('Warning: Please check your credentials and try to connect again!')
    del password

BusinessCollection = FBW.collection('Businesses')

Successfully connected to the ArangoDB


In [21]:
#Required Functions

def fix_http(URL):
    if URL != '':
        if('http' in URL) & (URL[-1:] == '/'):
            return URL
        elif ('http' in URL) & (URL[-1:] != '/'):
            return URL + '/'
        elif ('http' not in URL) & (URL[-1:] == '/'):
            return 'http://' + URL
        else:
            return 'http://' + URL + '/'

ua = UserAgent()
def get_html(URL,Timeout):
    header = {'User-Agent':str(ua.random)}
    try:
        page = requests.get(URL, timeout=Timeout, headers=header)
    except:
        return None
    return page.text

def get_html_ssl(URL,Timeout):
    header = {'User-Agent':str(ua.random)}
    try:
        page = requests.get(URL, timeout=Timeout, headers=header, verify = False)
    except:
        return None
    return page.text

In [ ]:
BusinessCollection["426019644"]

In [20]:
tqdm(disable=True, total=0)
if len(tqdm._instances) > 0:
    while len(tqdm._instances) > 0:
        tqdm._instances.pop().close()
    clear_output(wait=True)

cursor = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor, total=cursor.count()):
    if "Final_Website" in document:
        document["HTML_R1_NOV20"] = get_html(fix_http(document["Final_Website"]), 10)
        BusinessCollection.update(document)

3179it [1:00:59,  1.15s/it]


In [ ]:
for document in tqdm(cursor, total=cursor.count()):
    if "Final_Website" in document:ar
        document["HTML_R1_NOV20"] = get_html(fix_http(document["Final_Website"]), 10)
        BusinessCollection.update(document)

In [8]:
count_nempty = 0
cursor2 = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor2, total=cursor2.count()):
    if "HTML_R1_NOV20" in document:
        if(document["HTML_R1_NOV20"] is not None):
            count_nempty += 1
count_nempty

3179it [00:03, 911.78it/s]


2920

In [9]:
print("Total Scraped Websites in NOV 20, 2020:".ljust(30), count_nempty)

Total Scraped Websites in NOV 20, 2020: 2920


In [17]:
#TRY AGAIN FOR UNSCRAPED - WITHOUT SSL 
cursor = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor, total=cursor.count()):
    if (("Final_Website" in document) and (document["HTML_R1_NOV20"] is None)):
        document["HTML_R2_NOV20"] = get_html(fix_http(document["Final_Website"]), 10)
        BusinessCollection.update(document)
        
### 8 New Found ###

3179it [04:02, 13.09it/s]


In [ ]:
#TRY AGAIN FOR UNSCRAPED - WITH SSL VERIFY OFF 
cursor = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor, total=cursor.count()):
    if (("Final_Website" in document) and (document["HTML_R1_NOV20"] is None)):
        document["HTML_R2_NOV20"] = get_html(fix_http(document["Final_Website"]), 10)
        BusinessCollection.update(document)
        
### 8 New Found ###

In [ ]:
cursor = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor, total=cursor.count()):
    if (("Final_Website" in document) and (document["HTML_R1_NOV20"] is None) and (document["HTML_R2_NOV20"] is None)):
        document["HTML_R2_NOV20"] = get_html_ssl(fix_http(document["Final_Website"]), 10)
        if (document["HTML_R2_NOV20"] is not None):
            document["SSL_Fail"] = True
        BusinessCollection.update(document)
        
### 35 New Found ###

In [27]:
count_n = 0 
cursor = FBW.aql.execute('FOR doc IN Businesses RETURN doc', ttl=5000)
for document in tqdm(cursor, total=cursor.count()):
    if (("HTML_R1_NOV20" in document) or ("HTML_R2_NOV20" in document)):
        if((document["HTML_R1_NOV20"] is not None) or ((document["HTML_R2_NOV20"] is not None))):
            count_n += 1 
print("Total unfound websites: ", 3179-count_n)

3179it [00:03, 885.34it/s]

Total unfound websites:  216
